In [1]:
import os

In [23]:
%pwd

'c:\\Users\\nikil\\Documents\\Project\\NLP Projects\\textsum1'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nikil\\Documents\\Project\\NLP Projects\\textsum1'

In [24]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    #data_path: Path
    tokenizer_name: Path


In [25]:
from src.nlp.constants import *
from src.nlp.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            #data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


In [33]:
import os
import tqdm as notebook_tqdm
from src.nlp.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
from pathlib import Path

In [39]:
data_dir = Path("C:/Users/nikil/Documents/Project/NLP Projects/textsum1/artifacts/data_ingestion")
train_path = data_dir / "train.csv"
test_path = data_dir / "test.csv"
validation_path = data_dir / "validation.csv"


In [40]:
data_files = {
    "train": str(train_path),
    "test": str(test_path),
    "validation": str(validation_path)
}

In [41]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files=data_files)


Generating train split: 287113 examples [00:15, 18178.79 examples/s]
Generating test split: 11490 examples [00:00, 17676.22 examples/s]
Generating validation split: 13368 examples [00:00, 17459.40 examples/s]


In [42]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['article'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['highlights'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        #dataset_samsum = load_from_disk(self.config.data_path)
        dataset1 = dataset
        dataset1 = dataset1.map(self.convert_examples_to_features, batched = True)
        dataset1.save_to_disk(os.path.join(self.config.root_dir,"dataset1"))

In [43]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-05-25 12:11:06,842: INFO: common: yaml file loaded from config\config.yaml]
[2024-05-25 12:11:06,845: INFO: common: yaml file loaded from params.yaml]
[2024-05-25 12:11:06,847: INFO: common: created directory at artifacts]
[2024-05-25 12:11:06,849: INFO: common: created directory at artifacts/data_transformation]


c:\Users\nikil\anaconda3\envs\pytorch\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/287113 [00:00<?, ? examples/s]c:\Users\nikil\anaconda3\envs\pytorch\Lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 13368/13368 [00:02<00:00, 5485.37 examples/s]
